# **Launch Sites Locations Analysis with Folium**


In [1]:
import folium
import wget
import pandas as pd

In [2]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [ ]:
spacex_df=pd.read_csv(spacex_csv_file)

In [4]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [4]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.shape
spacex_df.groupby(['Launch Site'], as_index=False).first()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
circle = folium.Circle(nasa_coordinate, 
                       radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [12]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

for loc in locations:
    folium.Circle(location=loc['location'], 
                 radius=100,
                 color='#FF0000',
                 fill=True).add_child(folium.Popup(loc['popup'])).add_to(site_map)
    folium.map.Marker(location=loc['location'],
                        icon=DivIcon(icon_size=(20,20),
                                    icon_anchor=(0,0),
                                    html='<div style="font-size=12, color=#FF0000;"><b>%s</b></div>' % loc['popup'], )).add_to(site_map)
site_map    

Now, we mark the success/failed launches for each site on the map


In [14]:
from folium.plugins import MarkerCluster
map_cite=folium.Map(location=nasa_coordinate, zoom_start=4)
marker_cluster = MarkerCluster().add_to(site_map)
for label,x,y,c, in zip(spacex_df['Launch Site'], spacex_df.Lat, spacex_df.Long, spacex_df['class']):
    if(c==0):
        folium.Circle(location=[x,y], 
                     radius=1000,
                     color='#FF0000',
                     fill=True).add_child(folium.Popup(label)).add_to(site_map)
        folium.Marker(
            location=[x,y],
            popup=label,
            icon=folium.Icon(color='red')).add_to(marker_cluster)
    elif(c==1):
        folium.Circle(location=[x,y], 
                     radius=1000,
                     color='#00FF00',
                     fill=True).add_child(folium.Popup(label)).add_to(site_map)
        folium.Marker(
            location=[x,y],
            popup=label,
            icon=folium.Icon(color='green')).add_to(marker_cluster)
site_map

In [26]:
marker_cluster1=MarkerCluster()

In [27]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [28]:
site_map.add_child(marker_cluster1)

for index, record in spacex_df.iterrows():
    marker=folium.Marker(location=[record.Lat, record.Long],
                        icon=folium.Icon(color='white', icon_color=record['marker_color']),
                        popup=record['Launch Site'])
    marker_cluster1.add_child(marker)

site_map

In [29]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [30]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [36]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [52]:
distance_marker=folium.Marker(location=[28.56367, -80.56794], 
                             icon=DivIcon(icon_size=(20,20),
                                         icon_anchor=(0,0),
                                         html='<div style="font-size:12; color:#0035FF;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
                                         )
                             )
site_map.add_child(distance_marker)

In [53]:
lines=folium.PolyLine(locations=[[launch_sites_df['Lat'].iloc[1], launch_sites_df['Long'].iloc[1]],
                                 [28.56367, -80.56794]], 
                      weight=2)
site_map.add_child(lines)

Locations proximity to various landmark positions

In [69]:
locate=[{'location': [28.57209, -80.58528], 'popup': 'Nasa Railroad'}, 
        {'location': [28.5635, -80.57082], 'popup': 'Samual Philips pkwy'}, 
        {'location': [28.1053, -80.64583], 'popup': 'Melbourne'}]
print(locate[0]['location'][1])
launch_sites_df['Long'].iloc[1]
len(locate)

-80.58528


3

In [82]:
rail=[]
parkway=[]
mel=[]
for i in range(len(launch_sites_df)):
    t=calculate_distance(launch_sites_df['Lat'].iloc[i], 
                         launch_sites_df['Long'].iloc[i], 
                         locate[0]['location'][0], 
                         locate[0]['location'][1])
    rail.append(t)
for i in range(len(launch_sites_df)):
    t=calculate_distance(launch_sites_df['Lat'].iloc[i], 
                         launch_sites_df['Long'].iloc[i], 
                         locate[1]['location'][0], 
                         locate[1]['location'][1])
    parkway.append(t)
for i in range(len(launch_sites_df)):
    t=calculate_distance(launch_sites_df['Lat'].iloc[i], 
                         launch_sites_df['Long'].iloc[i], 
                         locate[2]['location'][0], 
                         locate[2]['location'][1])
    mel.append(t)
print(rail)
print(parkway)
print(mel)
combine=[rail, parkway, mel]
min(combine[0])
for i, l in enumerate(locate):
    print(i)
    print(l['location'])

[1.3358310606984158, 1.2889569136704222, 6.020177579437779, 3825.9350203096956]
[0.6523200802410717, 0.5871215373147786, 7.510373796862431, 3827.592726176256]
[51.272400471295974, 51.37799781345409, 52.050613260293744, 3838.8393047675336]
0
[28.57209, -80.58528]
1
[28.5635, -80.57082]
2
[28.1053, -80.64583]


In [83]:
for index, l in enumerate(locate):
    folium.Marker(location=l['location'],
                 icon=DivIcon(icon_size=(20,20),
                             icon_anchor=(0,0),
                             html='<div style="font-size:12; color:#0035FF;"<b>%s</b></div>' % "{:10.2f} KM".format(min(combine[index]))
                             )
                 ).add_to(site_map)

In [84]:
site_map

In [95]:
folium.PolyLine(locations=[[launch_sites_df['Lat'].iloc[1], launch_sites_df['Long'].iloc[1]], 
                [locate[0]['location'][0], locate[0]['location'][1]]], weight=2).add_to(site_map)

folium.PolyLine(locations=[[launch_sites_df['Lat'].iloc[1], launch_sites_df['Long'].iloc[1]], 
                [locate[1]['location'][0], locate[1]['location'][1]]], weight=2).add_to(site_map)

folium.PolyLine(locations=[[launch_sites_df['Lat'].iloc[0], launch_sites_df['Long'].iloc[0]], 
                [locate[2]['location'][0], locate[2]['location'][1]]], weight=2).add_to(site_map)

In [96]:
site_map